In [ ]:
import pandas as pd
import numpy
from sklearn import model_selection
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report
from sklearn import feature_selection
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
import seaborn as sn
import os
import matplotlib.pyplot as plt
import numpy as np
from sklearn.naive_bayes import MultinomialNB
# Import modules for evaluation purposes
# Import libraries for predcton
from sklearn import metrics
from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score, roc_curve, auc, f1_score
import nltk

nltk.download('punkt')
nltk.download('stopwords')


from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score




import torch
torch.cuda.empty_cache()
import seaborn as sns
import json
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
import logging
logging.basicConfig(level=logging.ERROR)
from google.colab import output
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt

#GPU usage setup
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
import random
random.seed(1)
np.random.seed(1)
torch.cuda.manual_seed(1)
torch.manual_seed(1)




[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
# mount drive to access data
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:

%cd /content/drive/MyDrive/nvdrs_narratives/round_1_annotation_topics

%ls

In [ ]:
# Read the Sample first round csv file
#df = pd.read_csv("pet_sampled_dw_completed.csv")

df1 = pd.read_csv("eviction or move_sampled_sp_completed.csv")
df2 = pd.read_csv("eviction or move_sample_0_sp_completed.csv")
# Bind/concatenate rows from additional sample, make sure we preserve column names
df = pd.concat([df1, df2])

df.info()
topic_name = "eviction or move"

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100 entries, 0 to 49
Data columns (total 10 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   IncidentID                  100 non-null    int64 
 1   NarrativeLE                 60 non-null     object
 2   matched_term_x              82 non-null     object
 3   LE_CircumstancesOtherText   5 non-null      object
 4   NarrativeCME                57 non-null     object
 5   matched_term_y              76 non-null     object
 6   CME_CircumstancesOtherText  2 non-null      object
 7   topic_name                  100 non-null    object
 8   topic_relevant_annotation   100 non-null    int64 
 9   comments                    25 non-null     object
dtypes: int64(2), object(8)
memory usage: 8.6+ KB


In [ ]:
df['label'] = df['topic_relevant_annotation']
df.drop(['IncidentID','LE_CircumstancesOtherText','CME_CircumstancesOtherText','topic_name','comments'], axis=1)
del df['IncidentID'], df['LE_CircumstancesOtherText'],df['CME_CircumstancesOtherText'], df['topic_name'], df['comments'], df['topic_relevant_annotation']



In [ ]:
import pandas as pd
import numpy as np

# Assuming df is your existing DataFrame
# df = pd.DataFrame(...)  # your original DataFrame

def transform_dataframe(original_df):
    # Create an empty DataFrame with the desired structure
    transformed_df = pd.DataFrame(columns=['text', 'matched_term', 'label'])

    # Iterate over each row in the original DataFrame
    for idx, row in original_df.iterrows():
        # Logic for handling NarrativeLE and NarrativeCME
        if pd.isna(row['NarrativeLE']) or row['NarrativeLE'] == '':
            transformed_df = transformed_df.append({
                'text': row['NarrativeCME'],
                'matched_term': row['matched_term_y'],
                'label': row['label']
            }, ignore_index=True)
        elif pd.isna(row['NarrativeCME']) or row['NarrativeCME'] == '':
            transformed_df = transformed_df.append({
                'text': row['NarrativeLE'],
                'matched_term': row['matched_term_x'],
                'label': row['label']
            }, ignore_index=True)
        else:
            # Create two new rows if both columns have values
            transformed_df = transformed_df.append([
                {
                    'text': row['NarrativeLE'],
                    'matched_term': row['matched_term_x'],
                    'label': row['label']
                },
                {
                    'text': row['NarrativeCME'],
                    'matched_term': row['matched_term_y'],
                    'label': row['label']
                }
            ], ignore_index=True)

    return transformed_df

# Apply the transformation
transformed_df = transform_dataframe(df)

# Output the transformed DataFrame
transformed_df



,text,matched_term,label
0,"V was W/M/46. V was found at his residence, ex...",['moving'],1
1,V was W/M/46. V was found in his apartment in ...,['moving'],1
2,The V is a 60 year old White male. The V was f...,"['eviction', 'moved']",1
3,The V is a 60 year old White male. The V was f...,['evicted'],1
4,"The V, a 27 year old white male, died of a sel...",['moved'],1
...,...,...,...
112,State and local police were notified of a deat...,['moved'],1
113,V is a 28 year old white male who was found de...,['out of state'],0
114,PA 2019 Incident: 253 was merged with PA 2019 ...,['moved'],0
115,"V1 (92, white, female) and V2 (62, white, male...",['moved'],0


In [ ]:
df = transformed_df
cols = ['text', 'matched_term']
df['text'] = df[cols].apply(lambda row: '</s>'.join(row.values.astype(str)), axis=1)
df["label"] = df["label"].astype(int)
df

,text,matched_term,label
0,"V was W/M/46. V was found at his residence, ex...",['moving'],1
1,V was W/M/46. V was found in his apartment in ...,['moving'],1
2,The V is a 60 year old White male. The V was f...,"['eviction', 'moved']",1
3,The V is a 60 year old White male. The V was f...,['evicted'],1
4,"The V, a 27 year old white male, died of a sel...",['moved'],1
...,...,...,...
112,State and local police were notified of a deat...,['moved'],1
113,V is a 28 year old white male who was found de...,['out of state'],0
114,PA 2019 Incident: 253 was merged with PA 2019 ...,['moved'],0
115,"V1 (92, white, female) and V2 (62, white, male...",['moved'],0


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, make_scorer
from datetime import datetime
import time

skf = StratifiedKFold(n_splits=5)
X = df['text'].values
y = df['label'].values

metrics = []

skf = StratifiedKFold(n_splits=5)

for train_index, test_index in skf.split(X, y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    vect = CountVectorizer(ngram_range=(1, 3), max_features=10000, stop_words="english")
    X_train_dtm = vect.fit_transform(X_train)
    X_test_dtm = vect.transform(X_test)


# Define the parameter grids for Naive Bayes, Random Forest, and Logistic Regression
nb_param_grid = {
    'alpha': [0.1, 0.5, 1.0],  # Different alpha values for MultinomialNB
}

rf_param_grid = {
    'n_estimators': [100, 200, 300],  # Number of trees in the forest
    'max_depth': [None, 10, 20],  # Maximum depth of the trees
    'min_samples_split': [2, 5, 10],  # Minimum samples required to split an internal node
}

lr_param_grid = {
    'C': [0.01, 0.1, 1.0],  # Regularization parameter for Logistic Regression
}

# Create a list to store the results
results = []

# Define a custom scoring function to prioritize precision for the positive class
positive_precision_scorer = make_scorer(precision_score, labels=[1], average='binary')
positive_recall_scorer = make_scorer(recall_score, labels=[1], average='binary')
positive_f1_scorer = make_scorer(f1_score, labels=[1], average='binary')

# Define custom scoring functions for negative class
negative_precision_scorer = make_scorer(precision_score, labels=[0], average='binary')
negative_recall_scorer = make_scorer(recall_score, labels=[0], average='binary')
negative_f1_scorer = make_scorer(f1_score, labels=[0], average='binary')

# Perform grid search for Naive Bayes
nb_grid_search = GridSearchCV(MultinomialNB(), nb_param_grid, cv=5, scoring='accuracy', verbose=1)
start_time = time.time()
nb_grid_search.fit(X_train_dtm, y_train)
end_time = time.time()
duration = end_time - start_time

# Get the best hyperparameters and the corresponding model
best_nb_model = nb_grid_search.best_estimator_
best_nb_params = nb_grid_search.best_params_

# Evaluate the Naive Bayes model on the test set
nb_test_predictions = best_nb_model.predict(X_test_dtm)
nb_accuracy = accuracy_score(y_test, nb_test_predictions)
nb_positive_precision = precision_score(y_test, nb_test_predictions, labels=[1], average='binary')
nb_positive_recall = recall_score(y_test, nb_test_predictions, labels=[1], average='binary')
nb_positive_f1 = f1_score(y_test, nb_test_predictions, labels=[1], average='binary')
nb_negative_precision = precision_score(y_test, nb_test_predictions, labels=[0], average='binary')
nb_negative_recall = recall_score(y_test, nb_test_predictions, labels=[0], average='binary')
nb_negative_f1 = f1_score(y_test, nb_test_predictions, labels=[0], average='binary')
nb_macro_precision = (nb_positive_precision + nb_negative_precision) / 2
nb_macro_recall = (nb_positive_recall + nb_negative_recall) / 2
nb_macro_f1 = (nb_positive_f1 + nb_negative_f1) / 2

# Store the results for Naive Bayes
results.append({
    'Model': 'Multinomial Naive Bayes',
    'Hyperparameters': best_nb_params,
    'Accuracy': nb_accuracy,
    'Positive Precision': nb_positive_precision,
    'Positive Recall': nb_positive_recall,
    'Positive F1': nb_positive_f1,
    'Negative Precision': nb_negative_precision,
    'Negative Recall': nb_negative_recall,
    'Negative F1': nb_negative_f1,
    'Macro Precision': nb_macro_precision,
    'Macro Recall': nb_macro_recall,
    'Macro F1': nb_macro_f1,
    'Timestamp': datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
    'Duration (s)': duration
})

# Perform grid search for Random Forest
rf_grid_search = GridSearchCV(RandomForestClassifier(random_state=1), rf_param_grid, cv=5, scoring='accuracy', verbose=1)
start_time = time.time()
rf_grid_search.fit(X_train_dtm, y_train)
end_time = time.time()
duration = end_time - start_time

# Get the best hyperparameters and the corresponding model
best_rf_model = rf_grid_search.best_estimator_
best_rf_params = rf_grid_search.best_params_

# Evaluate the Random Forest model on the test set
rf_test_predictions = best_rf_model.predict(X_test_dtm)
rf_accuracy = accuracy_score(y_test, rf_test_predictions)
rf_positive_precision = precision_score(y_test, rf_test_predictions, labels=[1], average='binary')
rf_positive_recall = recall_score(y_test, rf_test_predictions, labels=[1], average='binary')
rf_positive_f1 = f1_score(y_test, rf_test_predictions, labels=[1], average='binary')
rf_negative_precision = precision_score(y_test, rf_test_predictions, labels=[0], average='binary')
rf_negative_recall = recall_score(y_test, rf_test_predictions, labels=[0], average='binary')
rf_negative_f1 = f1_score(y_test, rf_test_predictions, labels=[0], average='binary')
rf_macro_precision = (rf_positive_precision + rf_negative_precision) / 2
rf_macro_recall = (rf_positive_recall + rf_negative_recall) / 2
rf_macro_f1 = (rf_positive_f1 + rf_negative_f1) / 2

# Store the results for Random Forest
results.append({
    'Model': 'Random Forest',
    'Hyperparameters': best_rf_params,
    'Accuracy': rf_accuracy,
    'Positive Precision': rf_positive_precision,
    'Positive Recall': rf_positive_recall,
    'Positive F1': rf_positive_f1,
    'Negative Precision': rf_negative_precision,
    'Negative Recall': rf_negative_recall,
    'Negative F1': rf_negative_f1,
    'Macro Precision': rf_macro_precision,
    'Macro Recall': rf_macro_recall,
    'Macro F1': rf_macro_f1,
    'Timestamp': datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
    'Duration (s)': duration
})

# Perform grid search for Logistic Regression
lr_grid_search = GridSearchCV(LogisticRegression(max_iter=1000), lr_param_grid, cv=5, scoring='accuracy', verbose=1)
start_time = time.time()
lr_grid_search.fit(X_train_dtm, y_train)
end_time = time.time()
duration = end_time - start_time

# Get the best hyperparameters and the corresponding model
best_lr_model = lr_grid_search.best_estimator_
best_lr_params = lr_grid_search.best_params_

# Evaluate the Logistic Regression model on the test set
lr_test_predictions = best_lr_model.predict(X_test_dtm)
lr_accuracy = accuracy_score(y_test, lr_test_predictions)
lr_positive_precision = precision_score(y_test, lr_test_predictions, labels=[1], average='binary')
lr_positive_recall = recall_score(y_test, lr_test_predictions, labels=[1], average='binary')
lr_positive_f1 = f1_score(y_test, lr_test_predictions, labels=[1], average='binary')
lr_negative_precision = precision_score(y_test, lr_test_predictions, labels=[0], average='binary')
lr_negative_recall = recall_score(y_test, lr_test_predictions, labels=[0], average='binary')
lr_negative_f1 = f1_score(y_test, lr_test_predictions, labels=[0], average='binary')
lr_macro_precision = (lr_positive_precision + lr_negative_precision) / 2
lr_macro_recall = (lr_positive_recall + lr_negative_recall) / 2
lr_macro_f1 = (lr_positive_f1 + lr_negative_f1) / 2

# Store the results for Logistic Regression
results.append({
    'Model': 'Logistic Regression',
    'Hyperparameters': best_lr_params,
    'Accuracy': lr_accuracy,
    'Positive Precision': lr_positive_precision,
    'Positive Recall': lr_positive_recall,
    'Positive F1': lr_positive_f1,
    'Negative Precision': lr_negative_precision,
    'Negative Recall': lr_negative_recall,
    'Negative F1': lr_negative_f1,
    'Macro Precision': lr_macro_precision,
    'Macro Recall': lr_macro_recall,
    'Macro F1': lr_macro_f1,
    'Timestamp': datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
    'Duration (s)': duration
})

# Create a DataFrame from the results
results_df = pd.DataFrame(results)

# Print the results
print(results_df)

# Save the results to a CSV file
results_df.to_csv(f'nvdrs_{topic_name}_model_results.csv', index=False)

Fitting 5 folds for each of 3 candidates, totalling 15 fits
Fitting 5 folds for each of 27 candidates, totalling 135 fits
Fitting 5 folds for each of 3 candidates, totalling 15 fits
                     Model                                    Hyperparameters  \
0  Multinomial Naive Bayes                                     {'alpha': 1.0}   
1            Random Forest  {'max_depth': None, 'min_samples_split': 5, 'n...   
2      Logistic Regression                                         {'C': 1.0}   

   Accuracy  Positive Precision  Positive Recall  Positive F1  \
0  0.652174            0.750000              0.3     0.428571   
1  0.521739            0.000000              0.0     0.000000   
2  0.739130            0.833333              0.5     0.625000   

   Negative Precision  Negative Recall  Negative F1  Macro Precision  \
0            0.750000              0.3     0.428571         0.750000   
1            0.000000              0.0     0.000000         0.000000   
2            0.8

# Best performers

Bootsrap resampling best performers

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Placeholder for best hyperparameters (you can modify these values)
best_nb_params = {'alpha': 1.0}
best_rf_params = {'n_estimators': 200, 'max_depth': None, 'min_samples_split': 5}
best_lr_params = {'C': 1.0}

X = df['text'].values
y = df['label'].values

skf = StratifiedKFold(n_splits=5)

for train_index, test_index in skf.split(X, y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    vect = CountVectorizer(ngram_range=(1, 2), max_features=10000, stop_words="english")
    X_train_dtm = vect.fit_transform(X_train)
    X_test_dtm = vect.transform(X_test)

    # Train and predict using the models
    models = {
        "Multinomial Naive Bayes": MultinomialNB(**best_nb_params),
        "Random Forest": RandomForestClassifier(random_state=1, **best_rf_params),
        "Logistic Regression": LogisticRegression(max_iter=1000, **best_lr_params)
    }

    predictions = {}

    for model_name, model in models.items():
        model.fit(X_train_dtm, y_train)
        preds = model.predict(X_test_dtm)
        predictions[model_name] = preds

    # Convert predictions to DataFrame
    predicted_df = pd.DataFrame(predictions)
    predicted_df['True Label'] = y_test

    results = []

    # Bootstrap by randomly sampling from predicted_df
    for _ in range(1000):
        sample_df = predicted_df.sample(len(predicted_df), replace=True)

        for model_name in models.keys():
            preds = sample_df[model_name].values
            true_labels = sample_df['True Label'].values

            accuracy = accuracy_score(true_labels, preds)
            positive_precision = precision_score(true_labels, preds, labels=[1], average='binary')
            positive_recall = recall_score(true_labels, preds, labels=[1], average='binary')
            positive_f1 = f1_score(true_labels, preds, labels=[1], average='binary')
            negative_precision = precision_score(true_labels, preds, labels=[0], average='binary')
            negative_recall = recall_score(true_labels, preds, labels=[0], average='binary')
            negative_f1 = f1_score(true_labels, preds, labels=[0], average='binary')
            macro_precision = (positive_precision + negative_precision) / 2
            macro_recall = (positive_recall + negative_recall) / 2
            macro_f1 = (positive_f1 + negative_f1) / 2

            results.append({
                'Model': model_name,
                'Accuracy': accuracy,
                'Positive Precision': positive_precision,
                'Positive Recall': positive_recall,
                'Positive F1': positive_f1,
                'Negative Precision': negative_precision,
                'Negative Recall': negative_recall,
                'Negative F1': negative_f1,
                'Macro Precision': macro_precision,
                'Macro Recall': macro_recall,
                'Macro F1': macro_f1
            })

    results_df = pd.DataFrame(results)

    # Calculate mean and confidence intervals
    summary = results_df.groupby('Model').agg({
        metric: ['mean', lambda x: np.percentile(x, 2.5), lambda x: np.percentile(x, 97.5)]
        for metric in ['Accuracy', 'Positive Precision', 'Positive Recall', 'Positive F1', 'Negative Precision', 'Negative Recall', 'Negative F1', 'Macro Precision', 'Macro Recall', 'Macro F1']
    })

    print(summary)

summary

summary.to_csv(f"summary_models_ndvrs_{topic_name}_bag_of_words.csv")



                         Accuracy                       Positive Precision  \
                             mean <lambda_0> <lambda_1>               mean   
Model                                                                        
Logistic Regression      0.584333   0.416667   0.791667           0.529941   
Multinomial Naive Bayes  0.748667   0.583333   0.916667           0.776624   
Random Forest            0.666167   0.458333   0.833333           0.634337   

                                              Positive Recall             \
                        <lambda_0> <lambda_1>            mean <lambda_0>   
Model                                                                      
Logistic Regression       0.312500   0.761975        0.821437   0.571429   
Multinomial Naive Bayes   0.499038   1.000000        0.638090   0.307500   
Random Forest             0.362879   0.900000        0.643980   0.356548   

                                   Positive F1  ... Negative F1  \
       

In [ ]:
print(summary)

                         Accuracy                       Positive Precision  \
                             mean <lambda_0> <lambda_1>               mean   
Model                                                                        
Logistic Regression      0.731130   0.565217   0.913043           0.821362   
Multinomial Naive Bayes  0.600652   0.391304   0.782609           0.609894   
Random Forest            0.518348   0.304348   0.739130           0.000000   

                                              Positive Recall             \
                        <lambda_0> <lambda_1>            mean <lambda_0>   
Model                                                                      
Logistic Regression       0.428571        1.0        0.491659   0.199545   
Multinomial Naive Bayes   0.000000        1.0        0.190210   0.000000   
Random Forest             0.000000        0.0        0.000000   0.000000   

                                   Positive F1  ... Negative F1  \
       

# Save Models

In [ ]:
# Save models
# /content/drive/MyDrive/nvdrs_narratives/round_1_annotation_topics/breakup_lr_model.pkl
#/content/drive/MyDrive/nvdrs_narratives/round_1_annotation_topics/child_custody_loss_lr_model.pkl
# /content/drive/MyDrive/nvdrs_narratives/round_1_annotation_topics/divorce_lr_model.pkl
# /content/drive/MyDrive/nvdrs_narratives/round_1_annotation_topics/pet_loss_lr_model.pkl
# /content/drive/MyDrive/nvdrs_narratives/round_1_annotation_topics/social_isolation_rf_model.pkl
# /content/drive/MyDrive/nvdrs_narratives/round_1_annotation_topics/eviction_or_move_lr_model.pkl

# Save the best model
import pickle
pickle.dump(best_lr_model , open('eviction_or_move_lr_model.pkl','wb'))

# Loading model to compare the results
test_model = pickle.load(open('social_isolation_rf_model.pkl','rb'))

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Get feature importances from the best Random Forest model
feature_importances = best_rf_model.feature_importances_

# Get the names of the features (tokens)
feature_names = np.array(vect.get_feature_names_out())

# Sort feature importances in descending order
sorted_indices = np.argsort(feature_importances)[::-1]

# Get the top 35 tokens and their importances
top_tokens = feature_names[sorted_indices][:30]
top_importances = feature_importances[sorted_indices][:30]

# Create a bar chart
plt.figure(figsize=(12, 8))
plt.barh(range(len(top_tokens)), top_importances, align='center')
plt.yticks(range(len(top_tokens)), top_tokens)
plt.xlabel('Feature Importance (Gini Index)')
plt.ylabel('Token')
plt.title(f'Top 30 {topic_name} Tokens by Feature Importance (Random Forest)')
plt.gca().invert_yaxis()  # Invert y-axis to show the most important tokens at the top
plt.savefig(f"{topic_name}_feature_importance_plot.png", bbox_inches='tight')  # Save the figure before showing it
plt.show()



In [ ]:
# Calculate feature contributions
coefs = best_lr_model.coef_  # Coefficients for each feature
feature_names = vect.get_feature_names_out()

# Create a DataFrame to store feature contributions
contributions_df = pd.DataFrame({'Feature': feature_names, 'Coefficient': coefs[0]})

# Sort the DataFrame by the absolute value of coefficients to identify significant features
contributions_df['Absolute_Coefficient'] = np.abs(contributions_df['Coefficient'])
contributions_df = contributions_df.sort_values(by='Absolute_Coefficient', ascending=False)

# Select the top N significant features for the plot
top_features = contributions_df.head(30)

# Determine the color for each bar based on the coefficient value
colors = ['red' if coef > 0 else 'blue' for coef in top_features['Coefficient']]

# Plot the feature contributions
plt.figure(figsize=(12, 8))
plt.barh(top_features['Feature'], top_features['Coefficient'], align='center', color=colors)
plt.xlabel('Feature Contribution')
plt.title(f'Top 30 {topic_name} Feature Contributions (Logistic Regression)')
plt.gca().invert_yaxis()  # Invert y-axis to show the most significant features at the top
plt.savefig(f"{topic_name}_feature_contributions_plot.png", bbox_inches='tight')  # Save the figure before showing it
plt.show()
